In [1]:
import pandas as pd
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_row", 160)
pd.set_option('max_colwidth', 200)

In [2]:
a = pd.read_csv('accepted.csv')
a.head(3)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,246

In [3]:
a.shape

(2260701, 151)

In [5]:
#1. Remove all obserations with No Funded Amount First, How many:
print(len(a[a.funded_amnt.isnull()]))

33


In [6]:
drop = a[a.funded_amnt.isnull()].index
a.drop(drop, axis=0, inplace=True)
a.shape

(2260668, 151)

In [7]:
#No more NaN funded amounts: 
a[a.funded_amnt.isna()].shape

(0, 151)

In [8]:
#2. How many value types for "loan_status"?
print(a.loan_status.value_counts(dropna=False).sum()) #Total matches

a.loan_status.value_counts(dropna=False)

2260668


Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [9]:
#Create new column and map Charged Off = 0, Fully Paid = 1, Opposite from Imbalanced set because its evenly balanced. 
a['loan_status_bin'] = a['loan_status'].map({'Charged Off':0, 'Fully Paid':1})

In [10]:
#Quick Check:
a[['loan_status', 'loan_status_bin']].sample(10)

,loan_status,loan_status_bin
1538648,Current,NaN
1861906,Fully Paid,1.0
1718779,Fully Paid,1.0
832790,Current,NaN
1050490,Fully Paid,1.0
1314461,Charged Off,0.0
134740,Fully Paid,1.0
995124,Current,NaN
920999,Current,NaN
1892656,Charged Off,0.0


In [11]:
#Data Dictionary to review features: 
datadic = pd.read_excel('LCDataDictionary.xlsx')
datadic

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


In [12]:
#Reorder columns alphabetically to compare vs data dictionary
a.reindex(sorted(a.columns), axis=1).sample(5)

,acc_now_delinq,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,application_type,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,debt_settlement_flag,debt_settlement_flag_date,deferral_term,delinq_2yrs,delinq_amnt,desc,disbursement_method,dti,dti_joint,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,hardship_amount,hardship_dpd,hardship_end_date,hardship_flag,hardship_last_payment_amount,hardship_length,hardship_loan_status,hardship_payoff_balance_amount,hardship_reason,hardship_start_date,hardship_status,hardship_type,home_ownership,id,il_util,initial_list_status,inq_fi,inq_last_12m,inq_last_6mths,installment,int_rate,issue_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,loan_status_bin,max_bal_bc,member_id,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,next_pymnt_d,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_acc_6m,open_act_il,open_il_12m,open_il_24m,open_rv_12m,open_rv_24m,orig_projected_additional_accrued_interest,out_prncp,out_prncp_inv,payment_plan_start_date,pct_tl_nvr_dlq,percent_bc_gt_75,policy_code,pub_rec,pub_rec_bankruptcies,purpose,pymnt_plan,recoveries,revol_bal,revol_bal_joint,revol_util,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_earliest_cr_line,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_mths_since_last_major_derog,sec_app_num_rev_accts,sec_app_open_acc,sec_app_open_act_il,sec_app_revol_util,settlement_amount,settlement_date,settlement_percentage,settlement_status,settlement_term,sub_grade,tax_liens,term,title,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,url,verification_status,verification_status_joint,zip_code
1670282,0.0,2.0,HI,71.0,65000.0,NaN,Individual,6291.0,294.0,98.2,0.0,0.0,0.0,N,NaN,NaN,0.0,0.0,NaN,Cash,25.66,NaN,Sep-2000,9 years,General Manager,669.0,665.0,19200.0,19200.0,C,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RENT,99946631,61.0,w,1.0,1.0,0.0,665.49,14.99,Mar-2017,Mar-2019,679.0,675.0,665.49,Mar-2019,19200.0,Current,NaN,8392.0,NaN,140.0,198.0,23.0,11.0,0.0,NaN,NaN,98.0,11.0,23.0,NaN,11.0,NaN,Apr-2019,0.0,3.0,4.0,4.0,4.0,5.0,5.0,5.0,4.0,8.0,0.0,0.0,0.0,1.0,8.0,0.0,3.0,1.0,1.0,0.0,1.0,NaN,7383.42,7383.42,NaN,100.0,100.0,1.0,1.0,1.0,debt_consolidation,n,0.0,17456.0,NaN,94.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C4,0.0,36 months,Debt consolidation,0.0,44039.0,61886.0,10.0,44039.0,26583.0,15900.0,1.0,43486.0,15955.770000,15955.77,4139.19,0.0,11816.58,18400.0,https://lendingclub.com/browse/loanDetail.action?loan_id=99946631,Verified,NaN,968xx
7477,0.0,2.0,FL,57.0,65000.0,NaN,Individual,14298.0,5390.0,36.0,0.0,0.0,0.0,N,NaN,NaN,1.0,0.0,NaN,Cash,13.20,NaN,Jun-2001,NaN,NaN,664.0,660.0,10000.0,10000.0,B,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MORTGAGE,67929915,55.0,w,0.0,1.0,1.0,318.79,9.17,Dec-2015,Dec-2018,719.0,715.0,318.75,Jan-2019,10000.0,Fully Paid,1.0,937.0,NaN,174.0,170.0,1.0,1.0,3.0,22.0,26.0,68.0,35.0,35.0,22.0,1.0,22.0,NaN,2.0,5.0,7.0,5.0,5.0,9.0,7.0,14.0,7.0,9.0,0.0,0.0,0.0,1.0,9.0,1.0,1.0,0.0,0.0,1.0,2.0,NaN,0.00,0.00,NaN,80.0,20.0,1.0,1.0,0.0,debt_consolidation,n,0.0,8548.0,NaN,59.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2,1.0,36 months,Debt consolidatio

In [13]:
#5a. Starting Point of (entire) Dataframe of accepted loans: 
a.shape

(2260668, 152)

In [14]:
a.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0

In [15]:
#5b. We will ONLY use loan_status_bin = 1 (Charged Off / Defaulted), or 0 (Paid In Full)

In [16]:
a.loan_status_bin.value_counts(dropna=False)

1.0    1076751
NaN     915358
0.0     268559
Name: loan_status_bin, dtype: int64

In [17]:
df = a[~a.loan_status_bin.isna()].reset_index(drop=True)
df.shape

(1345310, 152)

In [18]:
df.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0

In [19]:
#5c. Let's drop 5 obvious columns that won't help - 'loan_status', 'url', 'zip_code', 'id', 'member_id'
df = df.drop(['loan_status', 'url', 'zip_code', 'id', 'member_id'], axis=1)
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,NaN,debt_consolidation,Debt consolidation,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,n,NaN,small_business,Business,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truc

In [20]:
#5d. At time of application, we only know "loan_amnt", we do not know "funded_amnt" or "funded_amnt_inv". 
#5d. Drop 2 more columns, "funded_amnt" and "funded_amnt_inv". 

In [21]:
df = df.drop(['funded_amnt', 'funded_amnt_inv'], axis=1)
df

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,NaN,debt_consolidation,Debt consolidation,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,n,NaN,small_business,Business,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,n,NaN,home_impro

In [28]:
#5e. All the remaining loans do NOT have payment plan....let's drop it. 
df.pymnt_plan.value_counts()

n    1345310
Name: pymnt_plan, dtype: int64

In [29]:
df = df.drop(['pymnt_plan'], axis=1)
df

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,desc,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,NaN,debt_consolidation,Debt consolidation,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,NaN,small_business,Business,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,NaN,home_improvement,NaN,IL,10.

In [22]:
#6. Extract all loans with grade "G" (riskiest):
G = df[df.grade=="G"]
G

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
305,30000.0,60 months,27.49,924.87,G,G2,Service Technian Lead,1 year,MORTGAGE,115000.0,Verified,Dec-2015,n,NaN,other,Other,CO,17.73,0.0,Oct-2001,675.0,679.0,2.0,NaN,NaN,14.0,0.0,50634.0,58.7,20.0,w,0.0,0.0,13015.230000,13015.23,2934.76,7124.27,0.0,2956.20,532.1160,Dec-2016,924.87,NaN,Jun-2017,534.0,530.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,300130.0,3.0,3.0,2.0,3.0,1.0,20011.0,82.0,4.0,6.0,21644.0,64.0,86200.0,3.0,1.0,7.0,9.0,23087.0,31947.0,54.5,0.0,0.0,58.0,170.0,6.0,1.0,4.0,7.0,NaN,1.0,NaN,0.0,5.0,6.0,7.0,8.0,4.0,10.0,12.0,6.0,14.0,0.0,0.0,0.0,6.0,100.0,28.6,0.0,0.0,348717.0,70645.0,70200.0,24517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
337,19225.0,60 months,26.99,586.93,G,G1,OTR Truck Driver,< 1 year,MORTGAGE,56600.0,Verified,Dec-2015,n,NaN,debt_consolidation,NaN,OH,18.21,0.0,Apr-1990,710.0,714.0,1.0,51.0,28.0,8.0,1.0,7055.0,44.7,22.0,w,0.0,0.0,14014.250000,14014.25,4846.64,9167.61,0.0,0.00,0.0000,Jan-2018,586.93,NaN,Mar-2019,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,125.0,33946.0,5.0,3.0,3.0,3.0,3.0,26891.0,93.0,2.0,2.0,605.0,76.0,15800.0,2.0,4.0,5.0,5.0,4243.0,5847.0,12.7,0.0,0.0,122.0,308.0,2.0,2.0,0.0,2.0,NaN,0.0,51.0,0.0,2.0,4.0,3.0,5.0,10.0,5.0,12.0,4.0,8.0,0.0,0.0,0.0,5.0,90.5,33.3,0.0,1.0,44633.0,33946.0,6700.0,28833.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
701,17125.0,60 months,27.99,533.10,G,G3,Paralegal,4 years,RENT,45000.0,Source Verified,Dec-2015,n,NaN,debt_conso

In [23]:
G_total = G[['sub_grade', 'loan_status_bin']].groupby(by=['sub_grade']).count()
G_total = G_total.rename(columns={'loan_status_bin':'total loans'})
G_total

,total loans
sub_grade,
G1,2997
G2,2131
G3,1614
G4,1280
G5,1110


In [24]:
#Number of G loans that are "fully paid" by sub_grade
G_paid = G[G.loan_status_bin==0][['sub_grade', 'loan_status_bin']].groupby(by=['sub_grade']).count()
G_paid = G_paid.rename(columns={'loan_status_bin':'fully_paid'})
G_paid

,fully_paid
sub_grade,
G1,1434
G2,1047
G3,829
G4,655
G5,595


In [25]:
#Number of G loans that are "chargd off" by sub_grade
G_defaults = G[G.loan_status_bin!=0][['sub_grade', 'loan_status_bin']].groupby(by=['sub_grade']).count()
G_defaults = G_defaults.rename(columns={'loan_status_bin':'defaults'})
G_defaults

,defaults
sub_grade,
G1,1563
G2,1084
G3,785
G4,625
G5,515


In [26]:
G_combo = pd.concat([G_total, G_paid, G_defaults], axis=1)
G_combo

,total loans,fully_paid,defaults
sub_grade,,,
G1,2997,1434,1563
G2,2131,1047,1084
G3,1614,829,785
G4,1280,655,625
G5,1110,595,515


In [27]:
G_combo['default_ratio'] = G_combo['defaults']/G_combo['total loans']
G_combo

,total loans,fully_paid,defaults,default_ratio
sub_grade,,,,
G1,2997,1434,1563,0.521522
G2,2131,1047,1084,0.508681
G3,1614,829,785,0.486369
G4,1280,655,625,0.488281
G5,1110,595,515,0.463964


In [28]:
#Default ratio is approx 50/50 for each sub_grade of G. It's a coin toss. 

In [29]:
G.head(2)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_bin
305,30000.0,60 months,27.49,924.87,G,G2,Service Technian Lead,1 year,MORTGAGE,115000.0,Verified,Dec-2015,n,NaN,other,Other,CO,17.73,0.0,Oct-2001,675.0,679.0,2.0,NaN,NaN,14.0,0.0,50634.0,58.7,20.0,w,0.0,0.0,13015.23,13015.23,2934.76,7124.27,0.0,2956.2,532.116,Dec-2016,924.87,NaN,Jun-2017,534.0,530.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,300130.0,3.0,3.0,2.0,3.0,1.0,20011.0,82.0,4.0,6.0,21644.0,64.0,86200.0,3.0,1.0,7.0,9.0,23087.0,31947.0,54.5,0.0,0.0,58.0,170.0,6.0,1.0,4.0,7.0,NaN,1.0,NaN,0.0,5.0,6.0,7.0,8.0,4.0,10.0,12.0,6.0,14.0,0.0,0.0,0.0,6.0,100.0,28.6,0.0,0.0,348717.0,70645.0,70200.0,24517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
337,19225.0,60 months,26.99,586.93,G,G1,OTR Truck Driver,< 1 year,MORTGAGE,56600.0,Verified,Dec-2015,n,NaN,debt_consolidation,NaN,OH,18.21,0.0,Apr-1990,710.0,714.0,1.0,51.0,28.0,8.0,1.0,7055.0,44.7,22.0,w,0.0,0.0,14014.25,14014.25,4846.64,9167.61,0.0,0.0,0.000,Jan-2018,586.93,NaN,Mar-2019,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,125.0,33946.0,5.0,3.0,3.0,3.0,3.0,26891.0,93.0,2.0,2.0,605.0,76.0,15800.0,2.0,4.0,5.0,5.0,4243.0,5847.0,12.7,0.0,0.0,122.0,308.0,2.0,2.0,0.0,2.0,NaN,0.0,51.0,0.0,2.0,4.0,3.0,5.0,10.0,5.0,12.0,4.0,8.0,0.0,0.0,0.0,5.0,90.5,33.3,0.0,1.0,44633.0,33946.0,6700.0,28833.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [30]:
top_13 = ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
          'emp_length', 'annual_inc', 'verification_status', 'pymnt_plan',
          'dti', 'fico_range_low', 'loan_status_bin', 'home_ownership',
          'pub_rec']

In [31]:
G_top_13 = G[top_13].reset_index(drop=True)
G_top_13.head(5)

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,annual_inc,verification_status,pymnt_plan,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec
0,30000.0,60 months,27.49,924.87,G2,1 year,115000.0,Verified,n,17.73,675.0,0.0,MORTGAGE,0.0
1,19225.0,60 months,26.99,586.93,G1,< 1 year,56600.0,Verified,n,18.21,710.0,0.0,MORTGAGE,1.0
2,17125.0,60 months,27.99,533.10,G3,4 years,45000.0,Source Verified,n,17.44,660.0,1.0,RENT,1.0
3,11750.0,60 months,28.49,369.33,G4,1 year,28000.0,Verified,n,22.16,690.0,0.0,OWN,0.0
4,16275.0,60 months,27.49,501.74,G2,1 year,37000.0,Verified,n,18.72,685.0,0.0,RENT,0.0


In [32]:
G_top_13.dtypes

loan_amnt              float64
term                    object
int_rate               float64
installment            float64
sub_grade               object
emp_length              object
annual_inc             float64
verification_status     object
pymnt_plan              object
dti                    float64
fico_range_low         float64
loan_status_bin        float64
home_ownership          object
pub_rec                float64
dtype: object

In [33]:
G_top_13.isna().sum()

loan_amnt                0
term                     0
int_rate                 0
installment              0
sub_grade                0
emp_length             445
annual_inc               0
verification_status      0
pymnt_plan               0
dti                      6
fico_range_low           0
loan_status_bin          0
home_ownership           0
pub_rec                  0
dtype: int64

In [34]:
#Encoding Functions

def binary_temp(df, column, positive_value):
    temp_df = df.assign(new_be=df[column].apply(lambda x: 1 if x == positive_value else 0))
    return temp_df

def binary_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_be':column})
    return new_df
    
    #df[column + '_be'] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    #df_new = pd.concat([df, df[column].apply(lambda x: 1 if x == positive_value else 0)], axis=1)
    #df_new = df_new.drop(column, axis=1)
    #return df_new

def ordinal_temp(df, column, ordering):
    temp_df = df.assign(new_oe=df[column].apply(lambda x: ordering.index(x)))
    return temp_df

def ordinal_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_oe':column})
    return new_df

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new

In [35]:
#Return unique values of categorical columns:
{column: list(G_top_13[column].unique()) for column in G_top_13.columns if G_top_13.dtypes[column]=='object'}

{'term': [' 60 months', ' 36 months'],
 'sub_grade': ['G2', 'G1', 'G3', 'G4', 'G5'],
 'emp_length': ['1 year',
  '< 1 year',
  '4 years',
  '3 years',
  '10+ years',
  '5 years',
  '2 years',
  '8 years',
  '9 years',
  '6 years',
  '7 years',
  nan],
 'verification_status': ['Verified', 'Source Verified', 'Not Verified'],
 'pymnt_plan': ['n'],
 'home_ownership': ['MORTGAGE', 'RENT', 'OWN', 'ANY', 'NONE']}

In [36]:
#Check Binary Encoding:

temp = binary_temp(G_top_13, 'term', ' 60 months')

#for feature, positive_value in zip(binary_features, binary_positives):
#    df2 = binary_encode(df, feature, positive_value)

In [37]:
temp[['term', 'new_be']].sample(10)

,term,new_be
4795,60 months,1
8659,36 months,0
6404,60 months,1
8188,60 months,1
2756,60 months,1
5896,60 months,1
963,60 months,1
2968,36 months,0
8242,60 months,1
5533,60 months,1


In [38]:
#If ok, complete binary encoding:
new_df = binary_encode(temp, 'term')
new_df.head(5)

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,verification_status,pymnt_plan,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec,term
0,30000.0,27.49,924.87,G2,1 year,115000.0,Verified,n,17.73,675.0,0.0,MORTGAGE,0.0,1
1,19225.0,26.99,586.93,G1,< 1 year,56600.0,Verified,n,18.21,710.0,0.0,MORTGAGE,1.0,1
2,17125.0,27.99,533.10,G3,4 years,45000.0,Source Verified,n,17.44,660.0,1.0,RENT,1.0,1
3,11750.0,28.49,369.33,G4,1 year,28000.0,Verified,n,22.16,690.0,0.0,OWN,0.0,1
4,16275.0,27.49,501.74,G2,1 year,37000.0,Verified,n,18.72,685.0,0.0,RENT,0.0,1


In [39]:
grade_ordering = ['G1', 'G2', 'G3', 'G4', 'G5']

In [40]:
temp_df = ordinal_temp(new_df, 'sub_grade', grade_ordering)
temp_df[['sub_grade', 'new_oe']]

,sub_grade,new_oe
0,G2,1
1,G1,0
2,G3,2
3,G4,3
4,G2,1
...,...,...
9127,G1,0
9128,G2,1
9129,G2,1
9130,G2,1


In [41]:
#If ok, complete ordinal encoding:
new_df = ordinal_encode(temp_df, 'sub_grade')
new_df.head(5)

,loan_amnt,int_rate,installment,emp_length,annual_inc,verification_status,pymnt_plan,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec,term,sub_grade
0,30000.0,27.49,924.87,1 year,115000.0,Verified,n,17.73,675.0,0.0,MORTGAGE,0.0,1,1
1,19225.0,26.99,586.93,< 1 year,56600.0,Verified,n,18.21,710.0,0.0,MORTGAGE,1.0,1,0
2,17125.0,27.99,533.10,4 years,45000.0,Source Verified,n,17.44,660.0,1.0,RENT,1.0,1,2
3,11750.0,28.49,369.33,1 year,28000.0,Verified,n,22.16,690.0,0.0,OWN,0.0,1,3
4,16275.0,27.49,501.74,1 year,37000.0,Verified,n,18.72,685.0,0.0,RENT,0.0,1,1


In [42]:
new_df['emp_length'].replace(np.nan, "None", inplace=True)

In [43]:
new_df['emp_length'].value_counts()

10+ years    3048
2 years       814
3 years       738
< 1 year      736
1 year        616
5 years       570
4 years       536
6 years       450
None          445
7 years       432
8 years       401
9 years       346
Name: emp_length, dtype: int64

In [44]:
emp_ordering = [
    'None',
    '< 1 year',
    '1 year',
    '2 years',
    '3 years',
    '4 years',
    '5 years',
    '6 years',
    '7 years',
    '8 years',
    '9 years',
    '10+ years',
    ]

In [45]:
temp_df = ordinal_temp(new_df, 'emp_length', emp_ordering)
temp_df[['emp_length', 'new_oe']]

,emp_length,new_oe
0,1 year,2
1,< 1 year,1
2,4 years,5
3,1 year,2
4,1 year,2
...,...,...
9127,5 years,6
9128,2 years,3
9129,< 1 year,1
9130,10+ years,11


In [46]:
new_df = ordinal_encode(temp_df, 'emp_length')
new_df.head(3)

,loan_amnt,int_rate,installment,annual_inc,verification_status,pymnt_plan,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec,term,sub_grade,emp_length
0,30000.0,27.49,924.87,115000.0,Verified,n,17.73,675.0,0.0,MORTGAGE,0.0,1,1,2
1,19225.0,26.99,586.93,56600.0,Verified,n,18.21,710.0,0.0,MORTGAGE,1.0,1,0,1
2,17125.0,27.99,533.10,45000.0,Source Verified,n,17.44,660.0,1.0,RENT,1.0,1,2,5


In [47]:
#Dummify nominal features:
numerical_df = onehot_encode(new_df, 'verification_status')
numerical_df

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified
0,30000.0,27.49,924.87,115000.0,n,17.73,675.0,0.0,MORTGAGE,0.0,1,1,2,0,0,1
1,19225.0,26.99,586.93,56600.0,n,18.21,710.0,0.0,MORTGAGE,1.0,1,0,1,0,0,1
2,17125.0,27.99,533.10,45000.0,n,17.44,660.0,1.0,RENT,1.0,1,2,5,0,1,0
3,11750.0,28.49,369.33,28000.0,n,22.16,690.0,0.0,OWN,0.0,1,3,2,0,0,1
4,16275.0,27.49,501.74,37000.0,n,18.72,685.0,0.0,RENT,0.0,1,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,6000.0,28.18,248.77,30000.0,n,1.92,660.0,1.0,RENT,0.0,0,0,6,0,0,1
9128,12600.0,28.88,399.03,122000.0,n,20.39,670.0,0.0,RENT,3.0,1,1,3,1,0,0
9129,13000.0,28.88,411.70,73000.0,n,6.92,700.0,1.0,RENT,0.0,1,1,1,0,0,1
9130,15875.0,28.88,502.75,41600.0,n,28.02,665.0,1.0,RENT,1.0,1,1,11,0,0,1


In [48]:
#For G loans, all of them don't have payment plan, so we will drop this feature. 
numerical_df.pymnt_plan.value_counts()

n    9132
Name: pymnt_plan, dtype: int64

In [49]:
numerical_df = numerical_df.drop('pymnt_plan', axis=1)
numerical_df

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,loan_status_bin,home_ownership,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified
0,30000.0,27.49,924.87,115000.0,17.73,675.0,0.0,MORTGAGE,0.0,1,1,2,0,0,1
1,19225.0,26.99,586.93,56600.0,18.21,710.0,0.0,MORTGAGE,1.0,1,0,1,0,0,1
2,17125.0,27.99,533.10,45000.0,17.44,660.0,1.0,RENT,1.0,1,2,5,0,1,0
3,11750.0,28.49,369.33,28000.0,22.16,690.0,0.0,OWN,0.0,1,3,2,0,0,1
4,16275.0,27.49,501.74,37000.0,18.72,685.0,0.0,RENT,0.0,1,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,6000.0,28.18,248.77,30000.0,1.92,660.0,1.0,RENT,0.0,0,0,6,0,0,1
9128,12600.0,28.88,399.03,122000.0,20.39,670.0,0.0,RENT,3.0,1,1,3,1,0,0
9129,13000.0,28.88,411.70,73000.0,6.92,700.0,1.0,RENT,0.0,1,1,1,0,0,1
9130,15875.0,28.88,502.75,41600.0,28.02,665.0,1.0,RENT,1.0,1,1,11,0,0,1


In [50]:
#Dummify nominal features:
numerical_df2 = onehot_encode(numerical_df, 'home_ownership')
numerical_df2

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,loan_status_bin,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
0,30000.0,27.49,924.87,115000.0,17.73,675.0,0.0,0.0,1,1,2,0,0,1,0,1,0,0,0
1,19225.0,26.99,586.93,56600.0,18.21,710.0,0.0,1.0,1,0,1,0,0,1,0,1,0,0,0
2,17125.0,27.99,533.10,45000.0,17.44,660.0,1.0,1.0,1,2,5,0,1,0,0,0,0,0,1
3,11750.0,28.49,369.33,28000.0,22.16,690.0,0.0,0.0,1,3,2,0,0,1,0,0,0,1,0
4,16275.0,27.49,501.74,37000.0,18.72,685.0,0.0,0.0,1,1,2,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,6000.0,28.18,248.77,30000.0,1.92,660.0,1.0,0.0,0,0,6,0,0,1,0,0,0,0,1
9128,12600.0,28.88,399.03,122000.0,20.39,670.0,0.0,3.0,1,1,3,1,0,0,0,0,0,0,1
9129,13000.0,28.88,411.70,73000.0,6.92,700.0,1.0,0.0,1,1,1,0,0,1,0,0,0,0,1
9130,15875.0,28.88,502.75,41600.0,28.02,665.0,1.0,1.0,1,1,11,0,0,1,0,0,0,0,1


In [51]:
#All features are now in numerical form. 
numerical_df2.dtypes

loan_amnt          float64
int_rate           float64
installment        float64
annual_inc         float64
dti                float64
fico_range_low     float64
loan_status_bin    float64
pub_rec            float64
term                 int64
sub_grade            int64
emp_length           int64
Not Verified         uint8
Source Verified      uint8
Verified             uint8
ANY                  uint8
MORTGAGE             uint8
NONE                 uint8
OWN                  uint8
RENT                 uint8
dtype: object

In [52]:
#Check if any missing values left, especially in numerical columns which were not transformed:
numerical_df2.isna().sum()  #6 values in dti. Let's drop them. 

loan_amnt          0
int_rate           0
installment        0
annual_inc         0
dti                6
fico_range_low     0
loan_status_bin    0
pub_rec            0
term               0
sub_grade          0
emp_length         0
Not Verified       0
Source Verified    0
Verified           0
ANY                0
MORTGAGE           0
NONE               0
OWN                0
RENT               0
dtype: int64

In [56]:
numerical_df2[numerical_df2.dti.isna()]

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,loan_status_bin,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
1898,35000.0,30.79,1149.42,0.0,NaN,730.0,1.0,0.0,1,0,0,1,0,0,0,1,0,0,0
2429,30000.0,30.84,986.14,0.0,NaN,670.0,0.0,0.0,1,1,0,1,0,0,0,0,0,0,1
3503,12225.0,30.79,524.28,0.0,NaN,665.0,1.0,1.0,0,0,0,1,0,0,0,1,0,0,0
4238,17600.0,28.99,558.55,0.0,NaN,660.0,0.0,0.0,1,4,0,1,0,0,0,0,0,0,1
6293,24850.0,30.79,816.09,0.0,NaN,710.0,0.0,0.0,1,0,0,1,0,0,0,0,0,0,1
8655,10000.0,30.79,328.41,0.0,NaN,675.0,0.0,0.0,1,0,0,1,0,0,0,0,0,1,0


In [58]:
numerical_df2[numerical_df2.annual_inc==0]

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,loan_status_bin,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
1898,35000.0,30.79,1149.42,0.0,NaN,730.0,1.0,0.0,1,0,0,1,0,0,0,1,0,0,0
2429,30000.0,30.84,986.14,0.0,NaN,670.0,0.0,0.0,1,1,0,1,0,0,0,0,0,0,1
3503,12225.0,30.79,524.28,0.0,NaN,665.0,1.0,1.0,0,0,0,1,0,0,0,1,0,0,0
4238,17600.0,28.99,558.55,0.0,NaN,660.0,0.0,0.0,1,4,0,1,0,0,0,0,0,0,1
6293,24850.0,30.79,816.09,0.0,NaN,710.0,0.0,0.0,1,0,0,1,0,0,0,0,0,0,1
8655,10000.0,30.79,328.41,0.0,NaN,675.0,0.0,0.0,1,0,0,1,0,0,0,0,0,1,0


In [66]:
numerical_df2.dti.describe()

count    9126.000000
mean       21.516503
std        16.290095
min         0.000000
25%        14.352500
50%        20.920000
75%        27.750000
max       999.000000
Name: dti, dtype: float64

In [78]:
#Drop these 6 observations for now, may revisit later....

numerical_df2 = numerical_df2.dropna().reset_index(drop=True)
numerical_df2

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,loan_status_bin,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
0,30000.0,27.49,924.87,115000.0,17.73,675.0,0.0,0.0,1,1,2,0,0,1,0,1,0,0,0
1,19225.0,26.99,586.93,56600.0,18.21,710.0,0.0,1.0,1,0,1,0,0,1,0,1,0,0,0
2,17125.0,27.99,533.10,45000.0,17.44,660.0,1.0,1.0,1,2,5,0,1,0,0,0,0,0,1
3,11750.0,28.49,369.33,28000.0,22.16,690.0,0.0,0.0,1,3,2,0,0,1,0,0,0,1,0
4,16275.0,27.49,501.74,37000.0,18.72,685.0,0.0,0.0,1,1,2,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,6000.0,28.18,248.77,30000.0,1.92,660.0,1.0,0.0,0,0,6,0,0,1,0,0,0,0,1
9122,12600.0,28.88,399.03,122000.0,20.39,670.0,0.0,3.0,1,1,3,1,0,0,0,0,0,0,1
9123,13000.0,28.88,411.70,73000.0,6.92,700.0,1.0,0.0,1,1,1,0,0,1,0,0,0,0,1
9124,15875.0,28.88,502.75,41600.0,28.02,665.0,1.0,1.0,1,1,11,0,0,1,0,0,0,0,1


In [79]:
X = numerical_df2.drop('loan_status_bin', axis=1)
y = numerical_df2['loan_status_bin']

In [80]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
0,30000.0,27.49,924.87,115000.0,17.73,675.0,0.0,1,1,2,0,0,1,0,1,0,0,0
1,19225.0,26.99,586.93,56600.0,18.21,710.0,1.0,1,0,1,0,0,1,0,1,0,0,0
2,17125.0,27.99,533.10,45000.0,17.44,660.0,1.0,1,2,5,0,1,0,0,0,0,0,1
3,11750.0,28.49,369.33,28000.0,22.16,690.0,0.0,1,3,2,0,0,1,0,0,0,1,0
4,16275.0,27.49,501.74,37000.0,18.72,685.0,0.0,1,1,2,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,6000.0,28.18,248.77,30000.0,1.92,660.0,0.0,0,0,6,0,0,1,0,0,0,0,1
9122,12600.0,28.88,399.03,122000.0,20.39,670.0,3.0,1,1,3,1,0,0,0,0,0,0,1
9123,13000.0,28.88,411.70,73000.0,6.92,700.0,0.0,1,1,1,0,0,1,0,0,0,0,1
9124,15875.0,28.88,502.75,41600.0,28.02,665.0,1.0,1,1,11,0,0,1,0,0,0,0,1


In [81]:
y

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
9121    1.0
9122    0.0
9123    1.0
9124    1.0
9125    0.0
Name: loan_status_bin, Length: 9126, dtype: float64

In [82]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [83]:
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT
0,1.048421,-0.091987,0.896021,0.757086,-0.232455,-0.185394,-0.403489,0.446684,-0.356910,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,1.094521,-0.010468,-0.366949,-0.861164
1,-0.151844,-0.288444,-0.256434,-0.383695,-0.202987,1.630793,1.202374,0.446684,-1.079438,-1.449304,-0.255685,-0.707630,0.808060,-0.02094,1.094521,-0.010468,-0.366949,-0.861164
2,-0.385771,0.104469,-0.440006,-0.610289,-0.250258,-0.963760,1.202374,0.446684,0.365619,-0.422792,-0.255685,1.413168,-1.237531,-0.02094,-0.913641,-0.010468,-0.366949,1.161219
3,-0.984511,0.300926,-0.998500,-0.942366,0.039504,0.592972,-0.403489,0.446684,1.088147,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,2.725178,-0.861164
4,-0.480455,-0.091987,-0.546951,-0.766560,-0.171678,0.333516,-0.403489,0.446684,-0.356910,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,-1.625023,0.179123,-1.409638,-0.903298,-1.203036,-0.963760,-0.403489,-2.238718,-1.079438,-0.166164,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219
9122,-0.889826,0.454162,-0.897216,0.893823,-0.069156,-0.444850,4.414100,0.446684,-0.356910,-0.936048,3.911065,-0.707630,-1.237531,-0.02094,-0.913641,-0.010468,-0.366949,1.161219
9123,-0.845269,0.454162,-0.854009,-0.063339,-0.896085,1.111882,-0.403489,0.446684,-0.356910,-1.449304,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219
9124,-0.525012,0.454162,-0.543507,-0.676704,0.399252,-0.704305,1.202374,0.446684,-0.356910,1.116976,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219


In [84]:
scaled = pd.concat([X,y], axis=1)
scaled

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,pub_rec,term,sub_grade,emp_length,Not Verified,Source Verified,Verified,ANY,MORTGAGE,NONE,OWN,RENT,loan_status_bin
0,1.048421,-0.091987,0.896021,0.757086,-0.232455,-0.185394,-0.403489,0.446684,-0.356910,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,1.094521,-0.010468,-0.366949,-0.861164,0.0
1,-0.151844,-0.288444,-0.256434,-0.383695,-0.202987,1.630793,1.202374,0.446684,-1.079438,-1.449304,-0.255685,-0.707630,0.808060,-0.02094,1.094521,-0.010468,-0.366949,-0.861164,0.0
2,-0.385771,0.104469,-0.440006,-0.610289,-0.250258,-0.963760,1.202374,0.446684,0.365619,-0.422792,-0.255685,1.413168,-1.237531,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,1.0
3,-0.984511,0.300926,-0.998500,-0.942366,0.039504,0.592972,-0.403489,0.446684,1.088147,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,2.725178,-0.861164,0.0
4,-0.480455,-0.091987,-0.546951,-0.766560,-0.171678,0.333516,-0.403489,0.446684,-0.356910,-1.192676,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,-1.625023,0.179123,-1.409638,-0.903298,-1.203036,-0.963760,-0.403489,-2.238718,-1.079438,-0.166164,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,1.0
9122,-0.889826,0.454162,-0.897216,0.893823,-0.069156,-0.444850,4.414100,0.446684,-0.356910,-0.936048,3.911065,-0.707630,-1.237531,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,0.0
9123,-0.845269,0.454162,-0.854009,-0.063339,-0.896085,1.111882,-0.403489,0.446684,-0.356910,-1.449304,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,1.0
9124,-0.525012,0.454162,-0.543507,-0.676704,0.399252,-0.704305,1.202374,0.446684,-0.356910,1.116976,-0.255685,-0.707630,0.808060,-0.02094,-0.913641,-0.010468,-0.366949,1.161219,1.0


In [85]:
scaled.to_csv('G_12_feat_scaled.csv', index=False)